In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from hpe_library.lib_import import *
from hpe_library. my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

In [2]:
joint_2ds = {'train': [], 'test': []}
confidences = {'train': [], 'test': []}
joint3d_images = {'train': [], 'test': []}
joints_25d_images = {'train': [], 'test': []}
_25d_factors = {'train': [], 'test': []}
camera_names = {'train': [], 'test': []}
actions = {'train': [], 'test': []}
sources = {'train': [], 'test': []}
frames = {'train': [], 'test': []}
world_3ds = {'train': [], 'test': []}
cam_3ds = {'train': [], 'test': []}
cam_params = {'train': [], 'test': []}

# target 
pkl_folder = '/home/hrai/codes/MotionBERT/data/motion3d/fit3d_gt_pkl'
test_subject = ['s04', 's05', 's07', 's08', 's09', 's10', 's11']
only_test = False

blacklist = []

for item in tqdm(os.listdir(pkl_folder)):
    splited = item.split('.')[0].split('_') 
    subject = splited[0]
    cam_num = splited[1]
    action = '_'.join(splited[2:])
    
    # filtering
    #if action not in action_list: continue 
    #if int(cam_num) not in camera_list: continue
    if only_test:
        if subject not in test_subject: continue
    if subject in blacklist: continue
    #print(subject, cam_num, action, phase) 
    
    item_path = os.path.join(pkl_folder, item) 
    data = readpkl(item_path) 
    if subject in test_subject: data_type = 'test' 
    else: data_type = 'train' 

    joint_2ds[data_type] += list(np.array(data['joint_2d'])) 
    confidences[data_type] += list(np.array(data['confidence'])) 
    joint3d_images[data_type] += list(np.array(data['joint3d_image'])) 
    joints_25d_images[data_type] += list(np.array(data['joints_2.5d_image'])) 
    _25d_factors[data_type] += data['2.5d_factor'] 
    camera_names[data_type] += data['camera_name']  
    actions[data_type] += data['action'] 
    sources[data_type] += data['source'] 
    frames[data_type] += data['frame'] 
    world_3ds[data_type] += list(np.array(data['world_3d'])) 
    cam_3ds[data_type] += list(np.array(data['cam_3d'])/1000.0) # mm to m
    
data_total = {'train': {}, 'test': {}}

for data_type in ['train', 'test']:
    data_total[data_type]['joint_2d']          = copy.deepcopy(np.array(joint_2ds[data_type]))
    data_total[data_type]['confidence']        = copy.deepcopy(np.array(confidences[data_type]))
    data_total[data_type]['joint3d_image']     = copy.deepcopy(np.array(joint3d_images[data_type]))
    data_total[data_type]['joints_2.5d_image'] = copy.deepcopy(np.array(joints_25d_images[data_type]))
    data_total[data_type]['2.5d_factor']       = copy.deepcopy(np.array(_25d_factors[data_type]))
    data_total[data_type]['camera_name']       = copy.deepcopy(np.array(camera_names[data_type]))
    data_total[data_type]['action']            = copy.deepcopy(np.array(actions[data_type]))
    data_total[data_type]['source']            = copy.deepcopy(np.array(sources[data_type]))
    data_total[data_type]['frame']             = copy.deepcopy(np.array(frames[data_type]))
    data_total[data_type]['world_3d']          = copy.deepcopy(np.array(world_3ds[data_type]))
    data_total[data_type]['cam_3d']            = copy.deepcopy(np.array(cam_3ds[data_type]))
    data_total[data_type]['cam_param']         = copy.deepcopy(np.array(cam_params[data_type]))
    
if len(data_total['train']['joint_2d']) == 0:
    for key in data_total['test'].keys():
        data_total['train'][key] = copy.deepcopy(data_total['test'][key][0:243])

data_total['train']['joint_2d'].shape, data_total['test']['joint_2d'].shape  

  0%|          | 0/1504 [00:00<?, ?it/s]

100%|██████████| 1504/1504 [00:58<00:00, 25.51it/s]


((230368, 17, 2), (1548924, 17, 2))

In [3]:
save_path = os.path.join('/home/hrai/codes/MotionBERT', f'data/motion3d/fit3d_gt_tr_s03.pkl')
savepkl(data_total, save_path)